# Sentence Transformer 來做 Triplet 的分類任務 (Positive / Negative)
任務說明:
給定一段句子與 triplet，判斷 triplet 是否為 positive 或是 negative
1. Positive (0) 定義 : 與 distractor 相關的 triplet<br>
2. Negative (1) 定義 : 與 distractor 不相關的 triplet<br>

### GPU

In [1]:
!nvidia-smi

Tue Aug 29 02:26:12 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA TITAN RTX                On | 00000000:01:00.0 Off |                  N/A |
| 40%   38C    P8               20W / 280W|      1MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Weight and Bias (Assisting Metrics, Optional)

In [2]:
project_name = "test Sentence Transformer on MCQ reranker"
import os

os.environ["WANDB_PROJECT"] = project_name

### Import

In [3]:
from transformers import AutoTokenizer, AutoModel
import torch

2023-08-29 02:26:19.516966: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-29 02:26:19.680014: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-29 02:26:20.157900: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.0: cannot open shared object file: No such file or directory
2023-08-29 02:26:20.158019: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvrtc.so.11.0: cannot open shared object file: No such file or direc

## Prepare Data

In [4]:
import json
import os, sys
import fnmatch

In [11]:
def read_data(item):
    path = '../../../../data/mcq/reranker.{}.json'.format(item)
    with open(path) as f:
        data = json.load(f)
    return data

In [12]:
train = read_data('train')
test = read_data('test')

In [13]:
len(train), len(test)

(2321, 259)

In [14]:
train[0].keys()

dict_keys(['sentence', 'distractors', 'answer', 'triplets'])

In [16]:
# Initial Triplet with Sentence Transformer
print(test[0]['sentence'])
print(test[0]['distractors'])
print(test[0]['answer'])
test[0]['triplets'][:10]

**blank** is used to describe a chemical released by an animal that affects the behavior or physiology of animals of the same species
['enzyme', 'isolate', 'amino']
pheromone


[['isa', 'protease', 'enzyme', 2.0],
 ['relatedto', 'used', 'use', 2.0],
 ['isa', 'enzyme', 'catalyst', 2.0],
 ['relatedto', 'catalyst', 'enzyme', 1.0],
 ['relatedto', 'carbohydrate', 'animal', 1.0],
 ['relatedto', 'pheromone', 'chemical', 1.0],
 ['relatedto', 'protease', 'enzyme', 1.0],
 ['relatedto', 'animal', 'animals', 1.0],
 ['relatedto', 'affects', 'affect', 1.0],
 ['relatedto', 'pheromone', 'species', 1.0]]

Relation Dictionary

In [17]:
Relation_Dict ={
 'antonym': 'is the antonym of',
 'atlocation' : 'is at location of',
 'capableof': 'is capable of',
 'causes' : 'causes',
 'createdby': 'is created by',
 'desires': 'desires',
 'hasproperty': 'has property',
 'hassubevent': 'has subevent',
 'isa':'is a kind of',
 'madeof':'is made of',
 'notcapableof':'has not capable of',
 'notdesires': "does not desires",
 'partof':'is part of',
 'receivesaction':'is',
 'relatedto':'is related to',
 'usedfor':'is used for'
}

In [18]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(train, random_state=777, train_size=0.9)
len(train), len(valid)

(2088, 233)

In [19]:
def processData(data):
    
    sentences = []
    triplets = []
    labels = []
    answers = []
    for d in data:

        sentence = d['sentence']
        distractors = d['distractors']
        answer = d['answer']
        triplet = d['triplets']


        distractors = [dis.strip() for dis in distractors]
        sentence = sentence.replace('**blank**', answer)

        
        for each_triplet in triplet:
            rel, source, target, weight = each_triplet

            sentences.append(sentence)
            triplets.append('{} {} {}'.format(source,Relation_Dict[rel],target))

            if source == answer or target == answer or source in distractors or target in distractors:
                labels.append(0)
            else:
                labels.append(1)
            
              
        
    return sentences, triplets, labels

In [20]:
train_sent, train_triplet, train_label = processData(train)
valid_sent, valid_triplet, valid_label = processData(valid)
test_sent, test_triplet, test_label = processData(test)

In [21]:
len(train_sent), len(valid_sent), len(test_sent)

(37740, 4114, 4390)

統計訓練與測試資料分布

In [22]:
print('Train 資料分布 : Positive 有 {} 筆, Negative 有 {} 筆，大約有 {:.2f}% 為 Positive。'.format(train_label.count(0),train_label.count(1),train_label.count(0)/len(train_sent)*100))
print('Valid 資料分布 : Positive 有 {} 筆, Negative 有 {} 筆，大約有 {:.2f}% 為 Positive。'.format(valid_label.count(0),valid_label.count(1),valid_label.count(0)/len(valid_sent)*100))
print('Test 資料分布 : Positive 有 {} 筆, Negative 有 {} 筆，大約有 {:.2f}% 為 Positive。'.format(test_label.count(0),test_label.count(1),test_label.count(0)/len(test_sent)*100))

Train 資料分布 : Positive 有 8616 筆, Negative 有 29124 筆，大約有 22.83% 為 Positive。
Valid 資料分布 : Positive 有 923 筆, Negative 有 3191 筆，大約有 22.44% 為 Positive。
Test 資料分布 : Positive 有 1346 筆, Negative 有 3044 筆，大約有 30.66% 為 Positive。


## Tokenization

In [23]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')

In [24]:
train_encodings = tokenizer(train_sent, train_triplet, truncation=True, padding=True)
valid_encodings = tokenizer(valid_sent, valid_triplet, truncation=True, padding=True)
test_encodings = tokenizer(test_sent, test_triplet, truncation=True, padding=True)

In [25]:
print(train_encodings['input_ids'][0])
print(tokenizer.decode(train_encodings['input_ids'][0]))

[0, 2003, 7146, 23899, 1014, 2000, 19448, 2103, 4080, 2300, 6153, 3437, 1016, 2, 2, 8764, 2007, 3145, 2004, 7146, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
<s> in continuous reinforcement, the reinforcer follows every correct response. </s> </s> sustained is related to continuous </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pa

In [26]:
def add_labels(encodings, label):
    encodings.update({'labels': label})

In [27]:
add_labels(train_encodings, train_label)
add_labels(valid_encodings, valid_label)
add_labels(test_encodings, test_label)

In [28]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [29]:
print(train_encodings['labels'][0])

0


## 定義 Dataset，並轉換成 tensor 格式

In [30]:
from torch.utils import data
import torch

class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings

  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

  def __len__(self):
    return len(self.encodings.input_ids)

train_dataset = Dataset(train_encodings)
valid_dataset = Dataset(valid_encodings)
test_dataset = Dataset(test_encodings)

In [31]:
train_dataset[0]

{'input_ids': tensor([    0,  2003,  7146, 23899,  1014,  2000, 19448,  2103,  4080,  2300,
          6153,  3437,  1016,     2,     2,  8764,  2007,  3145,  2004,  7146,
             2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

## Fine-Tuning

### Load Sentence-Transformers model from HuggingFace Hub

In [25]:
from transformers import MPNetConfig, MPNetForSequenceClassification
config  = MPNetConfig.from_pretrained('sentence-transformers/all-mpnet-base-v2', num_labels = 2) # num_labels 設定類別數
model = MPNetForSequenceClassification.from_pretrained('sentence-transformers/all-mpnet-base-v2', config=config)

Some weights of the model checkpoint at sentence-transformers/all-mpnet-base-v2 were not used when initializing MPNetForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a

查看模型架構

In [26]:
print(model)

MPNetForSequenceClassification(
  (mpnet): MPNetModel(
    (embeddings): MPNetEmbeddings(
      (word_embeddings): Embedding(30527, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): MPNetEncoder(
      (layer): ModuleList(
        (0): MPNetLayer(
          (attention): MPNetAttention(
            (attn): MPNetSelfAttention(
              (q): Linear(in_features=768, out_features=768, bias=True)
              (k): Linear(in_features=768, out_features=768, bias=True)
              (v): Linear(in_features=768, out_features=768, bias=True)
              (o): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [30]:
from transformers import TrainingArguments, Trainer

In [31]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [32]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
   
    results = metric.compute(predictions=predictions, references=labels)

    return {'accuracy': results['accuracy']}

In [33]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [34]:
batch_size = 32
args = TrainingArguments(
    output_dir = "./results",
    save_strategy = "epoch",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    weight_decay=0.01,
    eval_accumulation_steps = 1,
    report_to="wandb" if os.getenv("WANDB_PROJECT") else "none"
)

In [35]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [33]:
trainer.train()

/user_data/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 69438
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 10850
  Number of trainable parameters = 109488002
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hankystyle. Use `wandb login --relogin` to force relogin


You're using a MPNetTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/user_data/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 7623
  Batch size = 64
  Num examples = 7623
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-1085
Configuration saved in ./results/checkpoint-1085/config.json
Model weights saved in ./results/checkpoint-1085/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1085/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1085/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-3] due to args.save_total_limit
/user_data/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 7623
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-2170
Configuration saved in ./results/checkpoint-2170/config.json
Model w

TrainOutput(global_step=10850, training_loss=0.0886934984004992, metrics={'train_runtime': 3663.7041, 'train_samples_per_second': 189.529, 'train_steps_per_second': 2.961, 'total_flos': 4.53179295641628e+16, 'train_loss': 0.0886934984004992, 'epoch': 10.0})

In [35]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 7623
  Batch size = 64
/user_data/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.30712223052978516,
 'eval_accuracy': 0.8836416109143382,
 'eval_runtime': 22.2692,
 'eval_samples_per_second': 342.311,
 'eval_steps_per_second': 5.389,
 'epoch': 10.0}

In [34]:
predictions, labels, metrics = trainer.predict(test_dataset)
print('test: ')
metrics

***** Running Prediction *****
  Num examples = 8339
  Batch size = 64
/user_data/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


test: 


{'test_loss': 0.3614237308502197,
 'test_accuracy': 0.8732461925890395,
 'test_runtime': 5.2905,
 'test_samples_per_second': 1576.222,
 'test_steps_per_second': 24.761}

In [37]:
trainer.save_model('../../../../saved_models/Reranker/sentence-transformer-for-mcq')

Saving model checkpoint to ./saved_models/sentence-transformer-mcq
Configuration saved in ./saved_models/sentence-transformer-mcq/config.json
Model weights saved in ./saved_models/sentence-transformer-mcq/pytorch_model.bin
tokenizer config file saved in ./saved_models/sentence-transformer-mcq/tokenizer_config.json
Special tokens file saved in ./saved_models/sentence-transformer-mcq/special_tokens_map.json


## Inference


In [26]:
sentence = test_sent[0]
triplet = test_triplet[0]
print('sentence = ',sentence)
print('triplet = ',triplet)

sentence =  pheromone is used to describe a chemical released by an animal that affects the behavior or physiology of animals of the same species
triplet =  pheromone is related to species


In [24]:
from transformers import MPNetConfig, MPNetForSequenceClassification
config  = MPNetConfig.from_pretrained('../../../../saved_models/Reranker/sentence-transformer-for-mcq', num_labels = 2)
model = MPNetForSequenceClassification.from_pretrained('../../../../saved_models/Reranker/sentence-transformer-for-mcq', config=config)

In [27]:
input_encodings = tokenizer([sentence], [triplet], truncation=True, padding=True)

In [36]:
input_dataset = Dataset(input_encodings)

In [37]:
input_dataset[0]

{'input_ids': tensor([    0,  6891, 10628,  8206,  2067,  2007,  2113,  2004,  6239,  1041,
          5076,  2211,  2015,  2023,  4115,  2012, 13535,  2000,  5252,  2034,
         16131,  2001,  4180,  2001,  2000,  2172,  2431,     2,     2,  6891,
         10628,  8206,  2067,  2007,  3145,  2004,  2431,     2]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}

## Validation 分布情況

In [38]:
predictions, labels, metrics = trainer.predict(valid_dataset)
print('valid: ')
metrics

***** Running Prediction *****
  Num examples = 7623
  Batch size = 64
You're using a MPNetTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/user_data/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


valid: 


{'test_loss': 0.30712223052978516,
 'test_accuracy': 0.8836416109143382,
 'test_runtime': 24.2069,
 'test_samples_per_second': 314.91,
 'test_steps_per_second': 4.957}

In [39]:
predictions = np.argmax(predictions, axis=-1)

In [40]:
from datasets import load_metric
accuracy_metric = load_metric("accuracy")

accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
print('accuracy : ', accuracy)

/tmp/ipykernel_3894/2714362684.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy_metric = load_metric("accuracy")


accuracy :  {'accuracy': 0.8836416109143382}


In [41]:
from sklearn.metrics import classification_report

for line in classification_report(labels, predictions).split('\n'):
  print(line)

              precision    recall  f1-score   support

           0       0.76      0.51      0.61      1373
           1       0.90      0.97      0.93      6250

    accuracy                           0.88      7623
   macro avg       0.83      0.74      0.77      7623
weighted avg       0.88      0.88      0.87      7623



In [42]:
# **產生 confusion matrix heatmap **
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
cf_matrix = confusion_matrix(labels, predictions)
print(cf_matrix)

[[ 703  670]
 [ 217 6033]]


In [43]:
matthews = matthews_corrcoef(labels, predictions)
print(matthews)

0.5630356514978342


### HeatMap

<Axes: xlabel='Predicted', ylabel='True Label'>

In [44]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

col = [ i for i in range(0,2)] #類別名稱

df_cm = pd.DataFrame(cf_matrix, columns=col, index = col)
df_cm.index.name = 'True Label'
df_cm.columns.name = 'Predicted'

plt.figure(figsize = (8,8))
sn.heatmap(df_cm, annot=True)

## Testing 分布情況 

In [45]:
predictions, labels, metrics = trainer.predict(test_dataset)
print('test: ')
metrics

***** Running Prediction *****
  Num examples = 8339
  Batch size = 64
/user_data/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


test: 


{'test_loss': 0.3614237308502197,
 'test_accuracy': 0.8732461925890395,
 'test_runtime': 5.3017,
 'test_samples_per_second': 1572.9,
 'test_steps_per_second': 24.709}

In [46]:
predictions = np.argmax(predictions, axis=-1)

In [47]:
from datasets import load_metric
accuracy_metric = load_metric("accuracy")

accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
print('accuracy : ', accuracy)

accuracy :  {'accuracy': 0.8732461925890395}


In [48]:
from sklearn.metrics import classification_report

for line in classification_report(labels, predictions).split('\n'):
  print(line)

              precision    recall  f1-score   support

           0       0.86      0.52      0.65      1870
           1       0.88      0.98      0.92      6469

    accuracy                           0.87      8339
   macro avg       0.87      0.75      0.79      8339
weighted avg       0.87      0.87      0.86      8339



In [49]:
# **產生 confusion matrix heatmap **
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
cf_matrix = confusion_matrix(labels, predictions)
print(cf_matrix)

[[ 971  899]
 [ 158 6311]]


In [50]:
matthews = matthews_corrcoef(labels, predictions)
print(matthews)

0.6032235729292296


<Axes: xlabel='Predicted', ylabel='True Label'>

In [51]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

col = [ i for i in range(0,2)] #類別名稱

df_cm = pd.DataFrame(cf_matrix, columns=col, index = col)
df_cm.index.name = 'True Label'
df_cm.columns.name = 'Predicted'

plt.figure(figsize = (8,8))
sn.heatmap(df_cm, annot=True)